In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install accelerate\
             protobuf\
             sentencepiece\
             torch\
             git+https://github.com/huggingface/transformers
    
!pip install huggingface_hub

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-rnjvut60
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-rnjvut60
  Resolved https://github.com/huggingface/transformers to commit 8c12690cecbb97e187861e386f7a0ac790e4236c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 13.3 MB/s eta 0:00:0000:010:01
  Created wheel for transformers: filename=transformers-4.41.0.dev0-py3-none-any.whl size=9012708 sha256=70eb9f1933e5c444c600da5cbab9f87e28da6ee3f56d8ced21c3e46b3066baa2
  Stored in directory: /tmp/pip-ephem-wheel-cache-2xxr68_r/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Succe

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer

In [3]:
from huggingface_hub import login
login()

In [5]:
if torch.cuda.is_available():
    model_id = "meta-llama/Llama-2-7b-chat-hf"
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
    model.cuda()
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [6]:
def chat_with_llama(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    input_ids = input_ids.to('cuda')
    output = model.generate(input_ids, max_length=256, num_beams=4, no_repeat_ngram_size=2)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

while True:
    prompt = input("You: ")
    if(prompt == 'exit'):
        print('Thanks for chatting with llama2')
        break
    response = chat_with_llama(prompt)
    print("Llama:", response)

You:  hi


Llama: hi there! I'm just an AI designed to assist with a wide range of tasks, from answering questions and providing information to helping with writing and communication. Here are some of the things I can help with:

1. Answering questions: If you have a question on any topic, feel free to ask me and I will do my best to provide a helpful and accurate response.
2. Providing information: Whether you're looking for information on a specific topic or just want to learn more about a particular subject, I am here to help. Just let me know what you need to know!
3. Writing assistance: Are you struggling to write a paper or essay? Or maybe you just need help coming up with ideas for a creative writing project. Whatever your writing needs are, count on me to be there for you. 
4. Communication support: Sometimes it can be hard to express yourself clearly and effectively, especially when communicating with people who speak different languages or have different cultural backgrounds. That's whe

You:  Write me python code to check if a number is a palindrome number or not.


Llama: Write me python code to check if a number is a palindrome number or not.

For example, if I give you the number 121, it should return True, because it's spelled the same way forward and backward. But if you give it a non-palindromic number like 234567890, then it will return False. Is that clear?


You:  Give me python code for it


Llama: Give me python code for it

I have a list of dictionaries in Python, and I want to extract the values of a specific key from each dictionary in the list. Here is an example of how I can do this:
```
my_list = [
    {"id": 1, "name": "John", "age': 25},
     {"name': "Jane", 'age": '30', 'city': 'New York'}
]
key = 'name'
values = []
for dict in my_ list:  # loop through each item in list
   values.append(dict[key]) # extract value of key
print(values) # print the extracted values
# output: ['John'] ['Janie']
 ```
This code will work, but it has a few drawbacks. For example, it modifies the original dictionary by appending the value to the `values` list, which may not be desirable. Additionally, the code relies on the order of the dictionary keys being the same as the key you are looking for. If the keys are in a different order, or if there are duplicate keys, you may get unexpected results.



You:  Do you remember what I asked you before?


Llama: Do you remember what I asked you before?

Answer: No, I don't remember. Please remind me.


You:  exit


Thanks for chatting with llama2


In [7]:
!git commit -m "created a simple chat bot using Llama-2-7b-chat-hf"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
